## Prédire le prix de l'immobilier
Le jeu de données décrit des biens avec des variables quantitatives et qualitatives ainsi que le prix de vente
Introduction à la régression linéaire et aux variantes (rég. pénalisée comme lasso)
Le prix de vente va etre modélisé par les autres variables

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
repertoire = "../data"
os.chdir(repertoire)

#https://www.kaggle.com/code/michaelfumery/exercice-pr-diction-de-prix-de-maison/notebook

### Chargement des données : apprentissage, appli, éch. de submission kaggle

In [ ]:
#sample_submission = pd.read_csv("prix_immo\\sample_submission.csv")

train = pd.read_csv("prix_immo\\train.csv") # jeu de données d'entrainement (avec le prix)
appli = pd.read_csv("prix_immo\\test.csv") # jeu de données où le prix est inconnu (à prédire)
#création d'une copie de chaque dataset
appli_copy  = appli.copy()
train_copy  = train.copy()

In [ ]:
# visualiser premieres lignes
train_copy.head()

In [ ]:
len(train_copy.columns)

il y a 81 colonnes dont 79 variables X (features), 1 colonne ID et la variable cible "SalePrice".
La variable ID pourrait être supprimée
Nous allons concaténer les 2 datasets train et appli afin de procéder aux memes transformations sur les deux jeux de données (une colonne est créée pours identifier Train vs appli)

In [ ]:
train_copy['train']  = 1
appli_copy['train']  = 0
data_full = pd.concat([train_copy, appli_copy], axis=0,sort=False)

In [ ]:
data_full.describe(include='all')

In [ ]:
data_full.info()

Nous allons calculer le pourcentage de valeurs manquantes pour chaque variable. On voit ci dessus que certaines var. ont un nombre non null faible. (MiscFeature)

In [ ]:
df_NULL = [(c, data_full[c].isna().mean()*100) for c in data_full]
df_NULL = pd.DataFrame(df_NULL, columns=["Colonne", "Taux de NULL"])
df_NULL.sort_values("Taux de NULL", ascending=False)

In [ ]:
# Variables avec plus de 10% de NULL : suppression

#df_NULL = df_NULL.drop(['SalePrice'],axis=0)

df_NULL = df_NULL[df_NULL["Taux de NULL"] > 10]

df_NULL = df_NULL[df_NULL["Colonne"] != 'SalePrice'] # on ne supprime pas la cible

df_NULL.sort_values("Taux de NULL", ascending=False)

In [ ]:
list_NULL_features = df_NULL["Colonne"].values

In [ ]:
#list_NULL_features = list(df_NULL.Colonne)

data_full = data_full.drop(list_NULL_features,axis=1)
data_full.shape

In [ ]:
df_NULL = [(c, data_full[c].isna().mean()*100) for c in data_full]
df_NULL = pd.DataFrame(df_NULL, columns=["Colonne", "Taux de NULL"])
df_NULL.sort_values("Taux de NULL", ascending=False)

### Feature Engineering : préparation des données pour la modélisation
On va traiter les val. manquantes
On traite differemment les var. numeriques (quantitatives) et les categorielles (qualitatives)

In [ ]:
categorical_features = data_full.select_dtypes(include=['object'])
numerical_features = data_full.select_dtypes(exclude=['object'])

In [ ]:
# Variables catégorielles :
print("Nombre de variables  :",categorical_features.shape[1])
print("\nNombre de valeurs nulles :\n",categorical_features.isnull().sum())

Après consultation de la description des fichiers de données, nous allons compléter les valeurs nulles des categoricla features ainsi :

    BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, GarageType, GarageFinish, GarageQual, FireplaceQu, GarageCond seront complétés avec la valeur "None",
    les autres variables avec leur propre valeur la plus fréquente.

In [ ]:
fill_None = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2', 'GarageType', 'GarageFinish', 'GarageQual','GarageCond']
categorical_features[fill_None]= categorical_features[fill_None].fillna('None')
fill_other = ['MSZoning','Utilities','Exterior1st','Exterior2nd','Electrical','KitchenQual','Functional','SaleType']
categorical_features[fill_other] = categorical_features[fill_other].fillna(categorical_features.mode().iloc[0])

In [ ]:
categorical_features.info()
# LEs var. catégorielles sont traitées : 2919 val. non nulles


nous allons pouvoir gérer les manquants sur les var. quantitatives:

Les numerical features (hors SalePrice) avec le plus grand nombre de NULL sont les suivantes :
    LotFrontage
    GarageYrBlt

Nous allons utiliser la médiane de ces variables pour compléter les valeurs nulles. Pour les autres variables, les valeurs nulles seront complétées à 0.

In [ ]:
numerical_features['GarageYrBlt'] = numerical_features['GarageYrBlt'].fillna(numerical_features['GarageYrBlt'].median())
#numerical_features['LotFrontage'] = numerical_features['LotFrontage'].fillna(numerical_features['LotFrontage'].median())
numerical_features['MasVnrArea'] = numerical_features['MasVnrArea'].fillna(numerical_features['MasVnrArea'].median())
numerical_features = numerical_features.fillna(0)


### Feature engineering : variables enrichies

In [ ]:

# Age de la maison
numerical_features['HouseAge'] = numerical_features['YrSold'] - numerical_features['YearBuilt']
# Age depuis la dernière rénovation    
numerical_features['RemodAge'] = numerical_features['YrSold'] - numerical_features['YearRemodAdd']


# Surface habitable totale
numerical_features['TotalSF'] = numerical_features['TotalBsmtSF']+ numerical_features['GrLivArea']+ numerical_features['GarageArea']
# Surface habitable au sol
numerical_features['1st2ndFlrSF'] = numerical_features['1stFlrSF'] + numerical_features['2ndFlrSF']   

# Lot Utilization: Ratio of total square footage to lot area
numerical_features['LotRatio'] = numerical_features['TotalSF'] / numerical_features['LotArea']


In [ ]:
# Garage Score: Quality * Condition of garage (numeric mapping)
qual_map = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0}
numerical_features['GarageScore'] = (categorical_features['GarageQual'].map(qual_map, na_action='ignore').fillna(0) * categorical_features['GarageCond'].map(qual_map, na_action='ignore').fillna(0))

categorical_features=categorical_features.drop(['GarageQual','GarageCond'], axis=1)

### Feature engineering : encoder les variables catégorielles
En effet, la modélisation suppose que les variables soient représentées par des nombres

In [ ]:

# pandas get dummies crée des variables binaires pour chaque modalité d'une variable catégorielle

categorical_features = pd.get_dummies(categorical_features, drop_first=True)

categorical_features.head()

### TABLE POUR L'APPRENTISSAGE (X, Y )
L'usage est de séparer les données en un jeu X contenant les features et un vecteur col y contenant juste la cible
Il est obligatoire en ML de séparer en un ech de train et de appli, le appli n'est pas utilisé dans l'apprentissage.

In [ ]:
df_final = pd.concat([numerical_features,categorical_features], axis=1,sort=False)
# Les va. manquantes sur le prix de vente ne peuvent pas etre imputées, il faut les supprimer
df_final = df_final[df_final['SalePrice'] >0 ]
df_final.shape

X = df_final.drop(['SalePrice','train','Id'], axis=1)
y = df_final['SalePrice']


In [ ]:
# Séparation en un jeu d'entrainement et un jeu d'application (test)
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=123)

### Modele naif : prédire le prix par la moyenne des prix sur l'apprentissage

In [ ]:
m1_prix_mean = np.mean(y_train)
m1_prix_mean

$$ RMSE = \sqrt{ \frac {\sum (obs - pred)^2 }{n} } $$

In [ ]:
RMSE_m1_train = np.sqrt(np.sum((y_train - m1_prix_mean)**2) / X_train.shape[0])
RMSE_m1_valid = np.sqrt(np.sum((y_valid - m1_prix_mean)**2) / X_valid.shape[0])

print(RMSE_m1_train,RMSE_m1_valid)
# le RMSE du modèle "naïf" qui prédit toujours la moyenne est de 80 k $ et 77 k $ sur le jeu de validation

In [ ]:
#Fonction de calculs des metriques importantes MAE, MSE, MAPE, RMSE
def metrics_regression(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    diff = y_true - y_pred
    mae = np.mean(abs(diff))
    mse = np.mean(diff**2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs(diff / y_true)) * 100
    dict_metrics = {"Métrique":["MAE", "MSE", "RMSE", "MAPE"], "Résultats":[mae, mse, rmse, mape]}
    df_metrics = pd.DataFrame(dict_metrics)
    return df_metrics

In [ ]:
# metrique sur l'échantillon de train validation
metrics_regression(y_train, m1_prix_mean)
metrics_regression(y_valid, m1_prix_mean)

### Modele de reg lineaire : selection de variables puis régression

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
m2_reglin = RFE(regressor, n_features_to_select=25, step=1) # step=1 means removing one feature at each iteration
m2_reglin = m2_reglin.fit(X_train, y_train)



In [ ]:
# The feature ranking, such that ranking_[i] corresponds to the ranking position of the i-th feature. Selected (i.e., estimated best) features are assigned rank 1.
ranking = m2_reglin.ranking_
print(ranking)

In [ ]:
len(m2_reglin.support_)

In [ ]:
# # Variables selectionnees
# mask = m2_reglin.support_ == True

# # Apply the mask
# X_train_df = X_train[mask]


In [ ]:
y_train_chap = m2_reglin.predict(X_train)
y_valid_chap = m2_reglin.predict(X_valid)

RMSE_m2_train = np.sqrt(np.sum((y_train - y_train_chap)**2) / X_train.shape[0])
RMSE_m2_valid = np.sqrt(np.sum((y_valid - y_valid_chap)**2) / X_valid.shape[0])

print(RMSE_m2_train,RMSE_m2_valid)

In [ ]:
# metrique sur l'échantillon de train validation
metrics_regression(y_valid, y_valid_chap)
# le MAPE est de 19 % sur le jeu de validation

In [ ]:
# Graphique des prédictions en fonction des valeurs réelles, échantillon de validation
df_graph=pd.concat([pd.Series(y_valid),pd.Series(y_valid_chap)], axis=1,sort=False)
df_graph.columns=['SalePrice','predictions']

In [ ]:
df_graph.plot.scatter(x='SalePrice', y='predictions');

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(df_graph.SalePrice, df_graph.predictions)
ax.plot([df_graph.predictions.min(), df_graph.predictions.max()], [df_graph.predictions.min(), df_graph.predictions.max()], color='r')
ax.set(xlabel='SalePrice', ylabel='Prédictions')
plt.title("Projection des prédictions en fonction des valeurs réelles", fontsize=20)
plt.show()

### Modele de reg lineaire lasso
Il s'agit d'une reg lin pénalisée : la fonction de cout est pénalisée afin de pouvoir traiter les var corrélée, le LASSO supprime des variables en les mettant à 0
Une régression pénalisée de type ridge (L2) permet de contraindre l'espace des coef estimés pour ne pas qu'ils prennent des valeurs contradictoires et très élevées,
Si la régression est de type lasso (L1) alors certains coefficients vont être annulés.
Le paramètre alpha contrôle cela : 
C = regularization strength; must be a positive float = higher values specify stronger regularization.

A noter que dans certaines classes, le paramètre est C=1/alpha 

Tester plusieurs valeurs de alpha 10,25,100 .. et regardez l'impact sur les coefficients

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)  # normalise l’éch d’apprentissage
X_train_norm = scaler.transform(X_train)
X_valid_norm= scaler.transform(X_valid)  # applique à l’éch test

In [ ]:
### Modele de reg lineaire lasso avec pénalisation 1/1000

from sklearn.linear_model import Lasso
from sklearn import linear_model
m3_reglinlasso1000=Lasso(alpha = 1000)
m3_reglinlasso1000 = m3_reglinlasso1000.fit(X_train_norm, y_train)


In [ ]:
# Coefficients du modèle
pd.DataFrame({'Coefficients': list(m3_reglinlasso1000.coef_)}, list(X_train.columns.values))

In [ ]:
# Combien sont nuls sur 233
coef=list(m3_reglinlasso1000.coef_)

# combien d'élemnts non nuls ?
feature_0_1000= list(map(lambda x: x==0.0, coef))
feature_non0_1000= list(map(lambda x: x!=0.0, coef))
print("Coef nuls",feature_0_1000.count(True)) 
print("% de coef nuls",feature_0_1000.count(True)/len(coef)*100) 

In [ ]:
### Modele de reg lineaire lasso avec pénalisation 1/1000
m3_reglinlasso5000=Lasso(alpha = 5000)
m3_reglinlasso5000 = m3_reglinlasso5000.fit(X_train_norm, y_train)

In [ ]:
# Combien sont nuls sur 233
coef=list(m3_reglinlasso5000.coef_)

# combien d'élemnts non nuls ?
feature_0= list(map(lambda x: x==0.0, coef))
print("Coef nuls",feature_0.count(True)) 
print("% de coef nuls",feature_0.count(True)/len(coef)*100) 

del feature_0

In [ ]:
y_train_chap = m3_reglinlasso5000.predict(X_train_norm)
y_valid_chap = m3_reglinlasso5000.predict(X_valid_norm)


print("RMSE Lasso 5000")

# metrique sur l'échantillon de train validation
metrics_regression(y_valid, y_valid_chap)

In [ ]:

y_train_chap = m3_reglinlasso1000.predict(X_train_norm)
y_valid_chap = m3_reglinlasso1000.predict(X_valid_norm)


print("RMSE Lasso 1000")


# metrique sur l'échantillon de train validation
metrics_regression(y_valid, y_valid_chap)
# le MAPE est de 10 % sur le jeu de validation pour la pénalisation 1000

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(y_valid, y_valid_chap)
ax.plot([y_valid_chap.min(), y_valid_chap.max()], [y_valid_chap.min(), y_valid_chap.max()], color='r')
ax.set(xlabel='SalePrice', ylabel='Prédictions')
plt.title("Projection des prédictions en fonction des valeurs réelles", fontsize=20)
plt.show()

## Faites une prédiction pour une nouvelle maison

In [ ]:
df_final = pd.concat([numerical_features,categorical_features], axis=1,sort=False)
new_house = df_final[df_final['SalePrice'] ==0 ]
new_house = new_house.drop(['SalePrice','train','Id'], axis=1).head(1)
new_house


In [ ]:
new_house_norm= scaler.transform(new_house)

prediction = m3_reglinlasso1000.predict(new_house_norm)
features = new_house.columns
print("\n La valeur prévue pour la maison est: {:,}".format(round(prediction[0])))

In [ ]:
np.array(new_house)

# D'autres models

In [ ]:
from sklearn.tree import DecisionTreeRegressor
modele_arbre=DecisionTreeRegressor(random_state = 42, max_depth = 5, min_samples_leaf = 30)
modele_arbre.fit(X_train_norm, y_train)
y_train_chap = modele_arbre.predict(X_train_norm)
y_valid_chap = modele_arbre.predict(X_valid_norm)


print("RMSE Lasso 5000")

# metrique sur l'échantillon de train validation
metrics_regression(y_valid, y_valid_chap)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

modele_random_forest=RandomForestRegressor(random_state = 42)
modele_random_forest.fit(X_train_norm, y_train)
y_train_chap = modele_random_forest.predict(X_train_norm)
y_valid_chap = modele_random_forest.predict(X_valid_norm)


print("RMSE Lasso 5000")

# metrique sur l'échantillon de train validation
metrics_regression(y_valid, y_valid_chap)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

modele_gradient_boosting = GradientBoostingRegressor(random_state = 42)
modele_gradient_boosting.fit(X_train_norm, y_train)
y_train_chap = modele_gradient_boosting.predict(X_train_norm)
y_valid_chap = modele_gradient_boosting.predict(X_valid_norm)


print("RMSE Lasso 5000")

# metrique sur l'échantillon de train validation
metrics_regression(y_valid, y_valid_chap)